In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws

In [2]:
import cloudknot as ck

In [19]:
def download_and_track(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    aws.download_s3(filename, op.split(filename)[1])
    
    outfile = 'Traj_' + op.split(filename)[1].split('.')[0] + '.csv'
    local_im = op.join(os.getcwd(), op.split(filename)[1])
    if not op.isfile(outfile):
        ij.track(local_im, outfile, template=None, fiji_bin=None, radius=4.5, threshold=0., 
              do_median_filtering=True, quality=4.5, median_intensity=300.0, snr=0.0, 
              linking_max_distance=7.0, gap_closing_max_distance=10.0, max_frame_gap=2,
              track_displacement=10.0)

        aws.upload_s3(outfile, op.join(op.split(filename)[0], outfile))

In [ ]:
# for i in range(0, 4):
#     for j in range(0, 4):

#         download_and_track('01_18_Experiment/P1/P1_S1_L_0000_{}_{}.tif'.format(i, j))

In [4]:
to_track = []
for i in range(0, 4):
    for j in range(0, 4):
        to_track.append('01_18_Experiment/P1/P1_S1_L_0000_{}_{}.tif'.format(i, j))

In [ ]:
to_track

In [5]:
github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
my_image = ck.DockerImage(func=download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

#Step here: delete user info from newly created docker image.

In [7]:
knot = ck.Knot(name='download_and_track_with_my_image_2',
               docker_image = my_image,
               pars_policies=('AmazonS3FullAccess',))

In [8]:
knot.map(to_track)

[<Future at 0x7f55844a7b50 state=running>,
 <Future at 0x7f55843cf0d0 state=running>,
 <Future at 0x7f5585f359d0 state=running>,
 <Future at 0x7f5585f35490 state=running>,
 <Future at 0x7f5585f35dd0 state=running>,
 <Future at 0x7f5585f35890 state=running>,
 <Future at 0x7f55861c2090 state=running>,
 <Future at 0x7f55861c2e90 state=running>,
 <Future at 0x7f55861c2f10 state=running>,
 <Future at 0x7f558590ead0 state=running>,
 <Future at 0x7f558610b650 state=running>,
 <Future at 0x7f558610be90 state=running>,
 <Future at 0x7f558610bed0 state=running>,
 <Future at 0x7f55858ded10 state=running>,
 <Future at 0x7f55858def50 state=running>,
 <Future at 0x7f55858de3d0 state=running>]

In [21]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
050e8039-4556-4314-9f62-a84d0e89d651        download_and_track_with_my_image_2-3        RUNNABLE 
d3d32ccb-04a5-4851-84e3-08703304588f        download_and_track_with_my_image_2-12        RUNNABLE 
5af780b2-16f8-439d-a4b1-55a5017c0b56        download_and_track_with_my_image_2-0        RUNNABLE 
c95a6d8b-7b6e-4884-9605-06c4089cd584        download_and_track_with_my_image_2-7        RUNNABLE 
06bde265-b03e-463a-8a3b-1d3e4adf6c04        download_and_track_with_my_image_2-1        RUNNABLE 
ce6cafb0-0375-41da-ac30-553db846b80d        download_and_track_with_my_image_2-10        RUNNABLE 
8c856ca8-289a-4f13-8ab2-ed5fc969aada        download_and_track_with_my_image_2-14        RUNNABLE 
ef6fdf63-426e-4dfa-b52e-bbc792f2b24e        download_and_track_with_my_image_2-9        RUNNABLE 
6c176bd5-bc53-4d65-a7b1-e78be0dc670c        download_and_track_with_my_image_2-15        RUNNABLE

In [ ]:
def download_and_split(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    local_name = op.split(filename)[1]
    DIR = op.split(filename)[0]
    try1 = filename.split('.')[0] + '_0_0.tif'
    try2 = filename.split('.')[0] + '_3_3.tif'
    
    s3 = boto3.client('s3')
    try:
        obj = s3.head_object(Bucket='ccurtis7.pup', Key=try1)
    except:
        try:
            obj = s3.head_object(Bucket='ccurtis7.pup', Key=try2)
        except:
            aws.download_s3(filename, local_name)
            names = ij.partition_im(local_name)
            for name in names:
                aws.upload_s3(name, op.join(op.split(filename)[0], name))

In [ ]:
pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
hemis = ["R", "L"]
videos = 15
folder = '01_18_Experiment'

In [ ]:
missing = []
for i in range(10, 15):
    missing.append("P1_S2_R_00{}.tif".format(i))

for i in range(10, 15):
    missing.append("P2_S3_L_00{}.tif".format(i))
    
for i in range(0, 15):
    missing.append("P3_S3_L_{}.tif".format("%04d" % i))

In [ ]:
to_partition = []
for pup in pups:
    for slic in slices:
        for hemi in hemis:
            for i in range(0, videos):
                name = "{}/{}/{}_{}_{}_{}.tif".format(folder, pup, pup, slic, hemi, "%04d" % i)
                if not name in missing:
                    to_partition.append(name)

In [ ]:
for name in to_partition:
    download_and_split(name)

In [ ]:
!rm P1_S1_R_0000*

In [ ]:
to_partition

In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3')

try:
    s3.Object("ccurtis7.pup", '01_18_Experiment/test_file.tif').load()

In [ ]:
s3 = boto3.client('s3')

result = s3.list_objects_v2(Bucket="ccurtis7.pup", Prefix="/01_18_Experiment/test_filebook.tif")

exists = False
if result:
    exists = True

print(exists)

In [ ]:
for obj in result.get('Contents', []):
    if obj['Key'] == "/01_18_Experiment/test_filebook.tif":
        print('Object here')
    else:
        print('Object not here')

In [ ]:
try:
    obj = s3.head_object(Bucket='ccurtis7.pup', Key="01_18_Experiment/test_file.tif")
except:
    print("Does not exist")